In [1]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup

In [2]:
from calendar import monthrange

In [3]:
month_day_tups = list()
months = [4, 5, 6, 7, 8, 9]

In [4]:
for month in months:
    days_in_month = monthrange(2016, month)[1]
    for i in range(days_in_month):
        month_day_tups.append((month, i + 1))

In [5]:
month_day_tups.append((10, 1))
month_day_tups.append((10, 2))
month_day_tups = month_day_tups[2:]

In [69]:
import pandas as pd

In [128]:
def scrape_mlb_vegas_lines(day, month, year):
    zero_pad = lambda x: str(x) if len(str(x)) > 1 else '0' + str(x)
    date = str(year) + zero_pad(month) + zero_pad(day)
    month_day = str(month) + zero_pad(day)
    money_line_base_url = 'http://www.sportsbookreview.com/betting-odds/mlb-baseball/?date={}'
    over_under_base_url = 'http://www.sportsbookreview.com/betting-odds/mlb-baseball/totals/?date={}'
    #scrape money lines 
    ml_soup = BeautifulSoup(urlopen(money_line_base_url.format(date)).read(), "lxml")
    game_data_list = list()
    game_data_header = 'month_day,home_team,away_team,home_pitcher,away_pitcher,home_ml,away_ml'
    for div in ml_soup.findAll('div', {'class': 'data'}):
        for game in div.findAll('div', {'class': 'event-holder'}):
            try:
                game_data = month_day + ', '
                for i, tag in enumerate(game.findAll()):
                    if i == 36:
                        away_team, away_pitcher = tag.text.split('-')
                    if i == 42:
                        home_team, home_pitcher = tag.text.split('-')
                    if i == 59:
                        away_ml = tag.text
                    if i == 61:
                        home_ml = tag.text
                gd = [month_day, home_team, away_team, 
                                       home_pitcher, away_pitcher, home_ml, away_ml[-4:]]
                gd = [''.join([i if ord(i) < 128 else '' for i in text]) for text in gd]
                game_data_list.append(gd)
            except:
                pass
    #scrape over/unders
    ou_soup = BeautifulSoup(urlopen(over_under_base_url.format(date)).read(), "lxml")
    ou_data_list = list()
    for div in ou_soup.findAll('div', {'class': 'data'}):
        for game_index, game in enumerate(div.findAll('div', {'class': 'event-holder'})):
            try:
                for i, tag in enumerate(game.findAll()):
#                     print(i, tag.text)
                    if i == 38:
                        ou_away_pitcher = tag.text.split('-')[1]
                    if i == 67:
                        over_under_text = tag.text.replace('+', '-').split('\xa0')[0]
                        try:
                            over_under = float(over_under_text)
                        except:
                            over_under = float(over_under_text[:-1]) + 0.5
                ou_data_list.append([ou_away_pitcher, over_under])
            except Exception as e:
                pass
    
    game_data_df = pd.DataFrame(game_data_list, columns=game_data_header.split(','))
    ou_df = pd.DataFrame(ou_data_list, columns=['away_pitcher', 'over_under'])
    ou_df.away_pitcher = ou_df.away_pitcher.apply(
        lambda x: ''.join([i if ord(i) < 128 else '' for i in x]))
                    
    return game_data_df.merge(ou_df, on='away_pitcher', how='left')

In [129]:
df = scrape_mlb_vegas_lines(10, 4, 2016)


In [130]:
df

,month_day,home_team,away_team,home_pitcher,away_pitcher,home_ml,away_ml,over_under
0,410,NYM,PHI,Harvey(R),Hellickson(R),-230,+210,6.5
1,410,WSH,MIA,Ross(R),Koehler(R),-153,+141,8.0
2,410,ATL,STL,Perez(R),Wainwright(R),+146,-159,7.5
3,410,SF,LAD,Cueto(R),Kazmir(L),-142,+131,7.5
4,410,COL,SD,Bettis(R),Shields(R),-131,+121,11.0
5,410,ARI,CHC,Miller(R),Arrieta(R),+187,-204,8.0
6,410,TOR,BOS,Estrada(R),Wright(R),-148,+136,9.5
7,410,BAL,TB,Worley(R),Odorizzi(R),-101,-107,8.0
8,410,KC,MIN,Volquez(R),Nolasco(R),-164,+151,8.5
9,410,LAA,TEX,Weaver(R),Perez(L),-125,+115,8.5


In [ ]:
''

In [41]:
mdtups = month_day_tups[0:99] + month_day_tups[103:]

In [58]:
from time import sleep 

In [131]:
lines2016 = pd.DataFrame()
for mdtup in mdtups:
    try:
        print('Scraping lines for {}'.format(mdtup))
        days_lines = scrape_mlb_vegas_lines(mdtup[1], mdtup[0], 2016)
        lines2016 = pd.concat([lines2016, days_lines], axis=1)
        sleep(2)
    except Exception as e:
        print("Unable to scrape lines for {} due to {}".format(mdtup, e))

Scraping lines for (4, 3)
Scraping lines for (4, 4)
Scraping lines for (4, 5)
Scraping lines for (4, 6)
Scraping lines for (4, 7)
Scraping lines for (4, 8)
Scraping lines for (4, 9)
Scraping lines for (4, 10)
Scraping lines for (4, 11)
Scraping lines for (4, 12)
Scraping lines for (4, 13)
Scraping lines for (4, 14)
Scraping lines for (4, 15)
Scraping lines for (4, 16)
Scraping lines for (4, 17)
Scraping lines for (4, 18)
Scraping lines for (4, 19)
Scraping lines for (4, 20)
Scraping lines for (4, 21)
Scraping lines for (4, 22)
Scraping lines for (4, 23)
Scraping lines for (4, 24)
Scraping lines for (4, 25)
Scraping lines for (4, 26)
Scraping lines for (4, 27)
Scraping lines for (4, 28)
Scraping lines for (4, 29)
Scraping lines for (4, 30)
Scraping lines for (5, 1)
Scraping lines for (5, 2)
Scraping lines for (5, 3)
Scraping lines for (5, 4)
Scraping lines for (5, 5)
Scraping lines for (5, 6)
Scraping lines for (5, 7)
Scraping lines for (5, 8)
Scraping lines for (5, 9)
Scraping lines fo

In [46]:
import pandas as pd 

In [60]:
vegas_lines_header = 'month_day,home_team,away_team,home_pitcher,away_pitcher,home_ml,away_ml,over_under'
lines_df = pd.DataFrame(lines2016, columns=vegas_lines_header.split(','))

In [133]:
clean_lines = lines_df[lines_df.month_day != '403']

In [136]:
clean_lines.over_under.isnull().sum()

0

In [137]:
clean_lines.to_csv('../mlb_data/vegas_lines/2016lines.csv', index=False)